# Beispiel 1 für die Energiesystemmodellierung eines kleinen Energiesystem

Dieses Jupyter Notebook führt durch ein Beispiel der Energiesystemmodellierung. Dabei werden die folgenden Schritte durchgeführt:
* [Import](#chapter1)
    * [Import der benötigten Python Module](#section_1_1)
    * [Import der Daten](#section_1_2)
* [Definition der Variablen und Objekte](#chapter2)
    * [Definition von Anlagenparametern](#section_2_1)
        * [Komponenten des Haushalts](#section_2_1_1)
        * [Netzanschluss](#section_2_1_2)
    * [Verwendung von PyPSA um das Energiemodell aufzubauen](#section_2_2)
        * [Definition des Netzwerks](#section_2_2_1)
        * [Knoten definieren](#section_2_2_2)
        * [Lasten definieren](#section_2_2_3)
        * [Erzeugungseinheiten definieren](#section_2_2_4)
        * [Speicher definieren](#section_2_2_5)
        * [Netzanschluss definieren](#section_2_2_6)
* [Simulation](#chapter3)
    * [Simulation der Basiskonfiguration](#section_3_1)
    * [Auswertungsfunktion](#section_3_2)
    * [Visualisierung der Ergebnisse](#section_3_3)
    * [Erweiterterung der Konfiguration](#section_3_4)
    * [Simulation der erweiterten Konfiguration](#section_3_5)
* [Visualisierung des Vergleichs](#chapter4)



# Import <a class="anchor" id="chapter1"></a>

## Import der benötigten Python Module <a class="anchor" id="section_1_1"></a>

In [ ]:
import pypsa
import pandas as pd
import math

## Import der Daten <a class="anchor" id="section_1_2"></a>

Die Werte der Zeitreihen sind in einem Unterordner "data" als csv-Datein abgelegt. Sie bilden den Oktober 2019 ab und werden mithilfe des Python Moduls pandas eingelesen und als Variablen angelegt. Dabei werden Parameter wie der Separator (hier ";") und die Indexspalte sowie die Funktion, aus der Indexspalte Zeit und Datenformate auszulesen gesetzt.

In [ ]:
electrical_load = pd.read_csv('./data/SumProfiles_Electricity.csv', sep = ';', index_col='Time', parse_dates=True)
thermal_load = pd.read_csv('./data/SumProfiles_Gas.csv', sep = ';', index_col='Time', parse_dates=True)
pv_infeed = pd.read_csv('./data/pv_infeed.csv', sep = ';', index_col='Time', parse_dates=True)

Die eingelesenen Zeitreihen sind so noch nicht vollständig nutzbar. Um die Einspeisung der PV-Anlage zu normieren wird eine neue Spalte mit dem normierten Leistungsoutput sowie neue Spalten mit der Leistung der Last in den jeweiligen Variablen angelegt.

In [ ]:
#Die gemessene PV-Einspeiseganglinie ist von einer Anlage mit einer Leistung von 3,6 kWp 
pv_infeed['p_max_pu'] = pv_infeed.power_kw/3.6 

#Das Profil hat eine Energienachfrage in kwh pro 10 Minuten, daher die Umrechnung in Leistung
electrical_load['power_kw'] = electrical_load.demand_kwh * (60.0/10.0)
thermal_load['power_kw'] = thermal_load.demand_kwh * (60.0/10.0)

Um einen kurzen Einblick in die Daten zu erhalten, schauen wir uns nur einen Ausschnitt aller drei Tabellen an:

In [ ]:
df_figure = pd.concat([electrical_load.power_kw, thermal_load.power_kw, pv_infeed.power_kw], axis = 1)
df_figure.columns = ['Elektrische Last', 'Thermische Last', 'PV Einspeisung']
df_figure.loc['10-13-19 00:00:00':'10-14-19 23:50:00'].plot(subplots = True, figsize=(15,10), title= 'Leistung in kW')

# Definition der Variablen und Objekte <a class="anchor" id="chapter2"></a>

## Definition von Anlagenparametern <a class="anchor" id="section_2_1"></a>

Die Parameter der PV-Anlage, sowie die Wärmepumpe und der Heizkessel mit thermischen Speicher werden hier definiert. Die Definition ist an reale technische Anlagen angelehnt. Um Einflüsse der verschiedenen Komponenten auf die Ergebnisse zu erfassen, können sie geändert werden.

### Komponenten des Haushalts <a class="anchor" id="section_2_1_1"></a>

Die Parameter sind in kW bzw. in cent/kWh angegeben und werden in ein python Dictionary gespeichert.

In [ ]:
# PV-Anlage
pv = {"electrical_power": 8.0}

#Heizkessel
heating_boiler = {"thermal_power": 20.0,
                   "efficiency":0.98,# Effizienz als normierte Kennzahl
                   "gas_costs": 6.2} #Kosten in cent/kWh

#Warmwasserspeicher
hot_water_storage =  {"thermal_power": 20.0,
                   "thermal_capacity":6.6, #Kapazität in kWh
                   "standing_loss": 0.01}  #Selbstentlasdungsverluste in Energie des Speicherinhalts, 
                                           #der sich nach einer Stunde selbst entladen hat.
                      

### Netzanschluss <a class="anchor" id="section_2_1_2"></a>

Der Netzanschluss wird für den Haushalt mit 14,5 kW angenommen. 
Hier ist auch eine Kappungsgrenze bei 70% der Nennleistung der PV-Anlage implementiert.

In [ ]:
# Leistungsbezug aus dem Netz
grid_import = {"electrical_power" : 14.5,
                "electricity_price": 30.0} #Preise in cent/kWh

#Leistungsabgabe in das Netz
grid_export = {"electrical_power" : 0.7 * pv["electrical_power"], #Repräsentiert die 70% Kappungsgrenze bei kleinen PV-Anlagen
                "electricity_revenue": 10.33} #EEG-Erlöse in cent/kWh

## Verwendung von PyPSA um das Energiemodell aufzubauen <a class="anchor" id="section_2_2"></a>

Im Folgenden verwenden wir das offene Python Modul PyPSA um unser Energiesystem zu bauen. Diese ist objektorientiert aufgebaut und hat verschiedene Module, welche zu einem Energiesystem zusammengesetzt werden können. Gleichzeitig kann der Einsatz der Komponenten nach Grenzkosten bestimmt werden. Es lassen sich auch AC- und DC-Netzflussberechnungen sowie Kapazitätsplanungen mit der Bibliothek ausführen.

### Definition des Netzwerks  <a class="anchor" id="section_2_2_1"></a>

Das Netzwerk ist das PyPSA Objekt, welches alle anderen Objekte enthält. Es definiert damit die Basisparameter unseres Energiesystems. Wir erstellen zunächst ein leeres Netzwerk und definieren dann die Zeitschritte. Diese lesen wir aus den Eingangsdaten ab.

In [ ]:
#Netzwerk definieren
network = pypsa.Network()
network.set_snapshots(pv_infeed.index)
network.snapshot_weightings = pd.Series(data = 1/6, index = network.snapshots) #Jeder Zeitpunkt repräsentiert 10 Minuten also 1/6 Stunde.

### Knoten definieren <a class="anchor" id="section_2_2_2"></a>

An Knoten werden bei PyPSA die Energieflüsse bilanziert und die Komponenten an diese angeschlossen. Wir definieren hier einen Strom- und einen thermischen Knoten.

In [ ]:
#Stromknoten definieren
network.add("Bus", 
            name = "electricity")

#Wärmeknoten definieren
network.add("Bus", 
            name = "thermal")

### Lasten definieren <a class="anchor" id="section_2_2_3"></a>

An jeden Knoten wird nun eine Last angeschlossen. Diese wird mit den Leistungszeitreihen der jeweiligen Lasten belegt.

In [ ]:
# Elektrische Last
network.add("Load", 
            name ="electricity_load", 
            bus = "electricity", 
            p_set = electrical_load.power_kw)

# Thermische last
network.add("Load", 
            name ="thermal_load", 
            bus = "thermal", 
            p_set = thermal_load.power_kw)


### Erzeugungseinheiten definieren <a class="anchor" id="section_2_2_4"></a>

In PyPSA sind Erzeugungseinheiten als "Generator" definiert. Diese haben eine installierte Leistung "p_nom" und ggf. eine normierte maximale Leistung "p_max_pu".
Wir definieren die PV-Anlage und den Heizkessel.

In [ ]:
#PV-Anlage
network.add("Generator", 
            name = "pv",
            bus = "electricity", 
            p_nom = pv["electrical_power"], 
            p_max_pu = pv_infeed.p_max_pu, 
            marginal_cost = 0)

#Heizkessel
network.add("Generator", 
            name = "boiler", 
            bus = "thermal", 
            p_nom = heating_boiler["thermal_power"], efficiency=heating_boiler["efficiency"], 
            marginal_cost = heating_boiler["gas_costs"]/heating_boiler["efficiency"])

### Speicher definieren <a class="anchor" id="section_2_2_5"></a>

Wir definieren den Speicher. Dabei wird ein Hilfsknoten definiert, an welchem die Einspeicherleistung, die Ausspeicherleistung als steuerbare Verbindung zwischen zwei Knoten und der Speicher selbst angeschlossen werden.

In [ ]:
# Hilfsknoten
network.add("Bus", 
            name = "storage_thermal")

#Einspeicherleistung vom Knoten "thermal" zum Knoten "storage_thermal"
network.add("Link", 
            name = "hot_water_storage_charge", 
            bus0 = "thermal", 
            bus1 = "storage_thermal", 
            p_nom = hot_water_storage["thermal_power"])

#Ausspeicherleistung vom Knoten "storage_thermal" zum Knoten "thermal"
network.add("Link", 
            name = "hot_water_storage_discharge", 
            bus0 = "storage_thermal", 
            bus1 = "thermal", 
            p_nom = hot_water_storage["thermal_power"])

#Speicher
network.add("Store", 
            name = "hot_water_storage", 
            bus = "storage_thermal", 
            e_nom = hot_water_storage["thermal_capacity"],
            e_cyclic = True, # Der Speicherfüllstand soll am Ende des Betrachtungszeitraumes wieder den Wert vom Anfang annehmen
            standing_loss = hot_water_storage["standing_loss"]
            )

### Netzanschluss definieren <a class="anchor" id="section_2_2_6"></a>

Der Netzanschluss wird hier auch als "Generator" definiert. Das heißt es kann Leistung aus dem Netz für einen bestimmten Preis bezogen werden. Die Netzeinspeisung wird auch als "Generator" definiert, allerdings wird die Leistungsabgabe dieses Generators mit einem Vorzeichenwechsel umgedreht. Der "Generator" kann also Leistung aufnehmen. Zusätzlich sind die Kosten negativ um Erlöse durch die EEG-Vergütung abzubilden. 

In [ ]:
#Netzbezug
network.add("Generator", 
            name = "grid_import",
            bus = "electricity", 
            p_nom = grid_import["electrical_power"], 
            marginal_cost = grid_import["electricity_price"])

#Netzeinspeisung
network.add("Generator", 
            name = "grid_export",
            bus = "electricity", 
            p_nom = grid_export["electrical_power"], 
            marginal_cost = -grid_export["electricity_revenue"], #Die Kosten müssen hier negativ sein, um Erlöse darzustellen
            sign = -1) #Das Vorzeichen -1 sorgt hier dafür, dass der Generator Leistung aufnimmt und nicht abgibt.

# Simulation <a class="anchor" id="chapter3"></a>

## Simulation der Basiskonfiguration <a class="anchor" id="section_3_1"></a>

Um die Basisvariante ohne Wärmepumpe zu berechnen, setzten wir die Leistung der Wärmepumpe temporär auf 0. 
Die in PyPSA implementierte Funktion lopf optimiert den Einsatz aller Komponenten so, dass die dabei entstehenden Kosten minimal sind.

In [ ]:
# Simulation der Basiskonfiguration für den kompletten Zeitraum:
network.lopf(solver_name = "glpk", pyomo = False) #Der Parameter pyomo = False hilft uns bei diesem Netzwerk RAM und Rechenzeit zu sparen.

## Auswertungsfunktion <a class="anchor" id="section_3_2"></a>

Funktionen werden in python üblicherweise am Anfang eines Scriptes definiert. Aus didaktischen Gründen kommt die Definition der Funktion erst jetzt. Für Aufgaben, welche wir mehrfach ausführen, lohnt sich die Definition einer Funktion. Diese kann dann im weiteren Verlauf immer wieder aufgerufen werden. 
Wir werten hier das gelöste Netzwerk nach den gewünschten Parametern aus. 

In [ ]:
#Auswertung der Kosten und Anteile
def cost_share_evaluation(network):
    '''
    This function evaluate the solved PyPSA network to get the costs, revenues and the share of the selfconsumption.
    
    Parameters
    ----------
    network:            PyPSA Network
                        The solved PyPSA network with all corresponding devices
    Returns
    -------
    results :           dict
                        The dictionary with all the results.
    '''
    
    grid_import = (network.generators_t.p['grid_import'] * network.snapshot_weightings).sum()
    grid_export = (network.generators_t.p['grid_export'] * network.snapshot_weightings).sum()
    self_consumed_pv = (network.generators_t.p['pv'] * network.snapshot_weightings).sum()- grid_export
    self_consumption_ratio = self_consumed_pv/(grid_import + self_consumed_pv)
    
    electricity_cost = grid_import * network.generators.marginal_cost['grid_import']
    electricty_revenues = grid_export * network.generators.marginal_cost['grid_export']
    gas_costs = (network.generators_t.p['boiler']*network.snapshot_weightings).sum() * network.generators.marginal_cost['boiler']
    
    curtailment = ((network.generators_t.p_max_pu['pv']*network.generators.p_nom['pv'] - network.generators_t.p['pv'])*network.snapshot_weightings).sum()

    results = {"Eigenverbrauchsanteil": self_consumption_ratio,
              "Stromkosten": electricity_cost,
              "Gaskosten": gas_costs,
              "Stromerlöse": -electricty_revenues,
              "Gesamtkosten": electricity_cost + gas_costs + electricty_revenues,
              "Nicht nutzbare Energie": curtailment}
    results_energy = {'Abregelung': curtailment,
                      'Netzeinspeisung': grid_export,
                      'Eigenverbrauch': self_consumed_pv,
                      'Netzbezug': -grid_import}
                      
    return results, results_energy

Ausführung der Auswertungsfunktion:

In [ ]:
results_base, results_energy_base  = cost_share_evaluation(network)

#Die Werte sollen auch als Text ausgegeben werden
print("Der Eigenverbrauchsanteil am Stromverbrauch in der Basiskonfiguration beträgt "
      + str(round(results_base["Eigenverbrauchsanteil"]*100,2))
      +"%. Dabei entstehen Stromkosten von "
      +str(round(results_base["Stromkosten"]/100,2))
      +"€, Gaskosten von "
      +str(round(results_base["Gaskosten"]/100,2))
      +"€ und EEG-Erlöse von "
      +str(round(results_base["Stromerlöse"]/100,2))
      +", wobei " +str(round(results_base["Nicht nutzbare Energie"]/100,1))+ "kWh PV-Strom abgeregelt werden muss."
      +" Die Gesamtkosten betragen "+ str(round(results_base["Gesamtkosten"]/100,2)) +"€."
     )

## Visualisierung der Ergebnisse <a class="anchor" id="section_3_3"></a>

In [ ]:
load_figure = network.loads_t.p_set['electricity_load']
load_figure.name = 'Elektrische Last'
ax = load_figure.loc['10-13-19 00:00:00':'10-14-19 23:50:00'].plot(legend = True, color = 'red')
df_figure_base = pd.concat([network.generators_t.p['pv'], network.generators_t.p['grid_import'], -network.generators_t.p['grid_export']], axis = 1)
df_figure_base.columns = ['PV Erzeugung', 'Netzbezug', 'Netzeinspeisung']
df_figure_base.loc['10-13-19 00:00:00':'10-14-19 23:50:00'].plot(legend =True, subplots = False, figsize=(15,10), kind= 'area', linewidth=0, ax=ax)


## Erweiterterung der Konfiguration<a class="anchor" id="section_3_4"></a>

Ein neues Netzwerk wird identisch zum Basisnetzwerk aufgebaut. Anschließend kann dieses neue Netzwerk um weitere Komponenten erweitert werden.

In [ ]:
# Basisnetzwerk implementieren:
network_modified= pypsa.Network()
network_modified.set_snapshots(pv_infeed.index)
network_modified.snapshot_weightings = pd.Series(data = 1/6, index = network.snapshots) #Jeder Zeitpunkt repräsentiert 10 Minuten also 1/6 Stunde.
#Stromknoten definieren
network_modified.add("Bus", 
            name = "electricity")
#Wärmeknoten definieren
network_modified.add("Bus", 
            name = "thermal")
# Elektrische Last
network_modified.add("Load", 
            name ="electricity_load", 
            bus = "electricity", 
            p_set = electrical_load.power_kw)
# Thermische last
network_modified.add("Load", 
            name ="thermal_load", 
            bus = "thermal", 
            p_set = thermal_load.power_kw)
#PV-Anlage
network_modified.add("Generator", 
            name = "pv",
            bus = "electricity", 
            p_nom = pv["electrical_power"], 
            p_max_pu = pv_infeed.p_max_pu, 
            marginal_cost = 0)
#Heizkessel
network_modified.add("Generator", 
            name = "boiler", 
            bus = "thermal", 
            p_nom = heating_boiler["thermal_power"], efficiency=heating_boiler["efficiency"], 
            marginal_cost = heating_boiler["gas_costs"]/heating_boiler["efficiency"])
# Hilfsknoten
network_modified.add("Bus", 
            name = "storage_thermal")
#Einspeicherleistung vom Knoten "thermal" zum Knoten "storage_thermal"
network_modified.add("Link", 
            name = "hot_water_storage_charge", 
            bus0 = "thermal", 
            bus1 = "storage_thermal", 
            p_nom = hot_water_storage["thermal_power"])
#Ausspeicherleistung vom Knoten "storage_thermal" zum Knoten "thermal"
network_modified.add("Link", 
            name = "hot_water_storage_discharge", 
            bus0 = "storage_thermal", 
            bus1 = "thermal", 
            p_nom = hot_water_storage["thermal_power"])
#Speicher
network_modified.add("Store", 
            name = "hot_water_storage", 
            bus = "storage_thermal", 
            e_nom = hot_water_storage["thermal_capacity"],
            e_cyclic = True, # Der Speicherfüllstand soll am Ende des Betrachtungszeitraumes wieder den Wert vom Anfang annehmen
            standing_loss = hot_water_storage["standing_loss"]           )
#Netzbezug
network_modified.add("Generator", 
            name = "grid_import",
            bus = "electricity", 
            p_nom = grid_import["electrical_power"], 
            marginal_cost = grid_import["electricity_price"])
#Netzeinspeisung
network_modified.add("Generator", 
            name = "grid_export",
            bus = "electricity", 
            p_nom = grid_export["electrical_power"], 
            marginal_cost = -grid_export["electricity_revenue"], #Die Kosten müssen hier negativ sein, um Erlöse darzustellen
            sign = -1) #Das Vorzeichen -1 sorgt hier dafür, dass der Generator Leistung aufnimmt und nicht abgibt.

Hier können nun neue Elemente in das Netzwerk intigriert werden.

In [ ]:
#Wärmepumpe
#Hier können Sie die Aufgabe mit der Wärmepumpe implementieren


In [ ]:
# Batteriespeicher
#Hier können Sie die Aufgabe mit dem Batteriespeicher implementieren


## Simulation der erweiterten Konfiguration <a class="anchor" id="section_3_5"></a>

In [ ]:
# Simulation für den kompletten Zeitraum:
network_modified.lopf(solver_name = "glpk", pyomo = False) #Der Parameter pyomo = False hilft uns bei diesem Netzwerk RAM und Rechenzeit zu sparen.

In [ ]:
# Ausführung der Auswertungsfunktion
results_with_modification, results_energy_with_modification = cost_share_evaluation(network_modified)
print("Der Eigenverbrauchsanteil am Stromverbrauch in der erweiterten Konfiguration beträgt "
      + str(round(results_with_modification["Eigenverbrauchsanteil"]*100,2))
      +"%.\n Dabei entstehen Stromkosten von "
      +str(round(results_with_modification["Stromkosten"]/100,2))
      +"€, Gaskosten von "
      +str(round(results_with_modification["Gaskosten"]/100,2))
      +"€ und EEG-Erlöse von "
      +str(round(results_with_modification["Stromerlöse"]/100,2))
      +", wobei " +str(round(results_with_modification["Nicht nutzbare Energie"]/100,1))
      + "kWh PV-Strom abgeregelt werden müssen.\n"
      +" Die Gesamtkosten betragen "+ str(round(results_with_modification["Gesamtkosten"]/100,2)) 
      +"€.\n"
      +"Gegenüber der Basiskonfiguration ist dies eine Erhöhung des Eigenverbrauchsanteils um "
      +str(round((results_with_modification["Eigenverbrauchsanteil"]- results_base["Eigenverbrauchsanteil"])*100,2))
      +" Prozentpunkte und eine Verringerung der Kosten um "
      +str(round((results_base["Gesamtkosten"]-results_with_modification["Gesamtkosten"])/100,2))
      +"€."
     )

# Visualisierung des Vergleichs <a class="anchor" id="chapter4"></a>

Wir visualisieren am Ende noch Ergebnisse der Simulation mit Wärmepumpe sowie einen Vergleich der Energiebilanzen mit und ohne Wärmepumpe. 

In [ ]:
total_load = pd.concat([network_modified.loads_t.p_set['electricity_load'], network_modified.links_t.p0['heat_pump']], axis = 1)
total_load.columns = ['Elektrische Last', 'Wärmepumpe']
ax = total_load.loc['10-13-19 00:00:00':'10-14-19 23:50:00'].plot(color = ['red', 'black'], legend = True, stacked = False)
df_figure_with_modification = pd.concat([network_modified.generators_t.p['pv'], network_modified.generators_t.p['grid_import'], -network_modified.generators_t.p['grid_export']], axis = 1)
df_figure_with_modification.columns = ['PV Erzeugung', 'Netzbezug', 'Netzeinspeisung']
df_figure_with_modification.loc['10-13-19 00:00:00':'10-14-19 23:50:00'].plot(subplots = False, figsize=(15,10), kind= 'area', linewidth=0, ax = ax)

In [ ]:
energy_figure = pd.DataFrame([results_energy_base, results_energy_with_modification], index = ['Basis', 'Mit Modifikation'])

Die Energiebilanz zeigt uns die Verwendung der Elektrizität im Monat Oktober mit und ohne Modifikation. Dabei können sehen, dass der Anteil des Eigenverbrauchs (in grün) mit zusätzlichen Flexibilitäten ansteigt.

In [ ]:
energy_figure.plot(kind='bar', stacked = True, figsize=(14,7), title = 'Elektrizitätsbilanz in kWh')